# Biopharma

### Install and import packages

In [ ]:
%reset -f
# Install and import packages
!pip install gurobipy
!pip install tabulate

import pandas as pd
import numpy as np
from gurobipy import Model, GRB, quicksum
from tabulate import tabulate
import datetime as dt
_empty_series = pd.Series(dtype=float)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 24.6 MB/s eta 0:00:00


### Raw Data

In [ ]:
selected_yr = 2023
base_yr = 1

demand = pd.DataFrame({
    'from': ['LatinAmerica', 'Europe', 'AsiaWoJapan', 'Japan', 'Mexico', 'U.S.'],
    'd_h': [  7, 15,  5,  7,  3, 18],
    'd_r': [  7, 12,  3,  8,  3, 17],
})
demand.set_index('from', inplace=True)

caps = pd.DataFrame({
    'plant': ['Brazil', 'Germany', 'India', 'Japan', 'Mexico', 'U.S.'],
    'cap': [18, 45, 18, 10, 30, 22],
})
caps.set_index('plant', inplace=True)

pcosts = pd.DataFrame({
    'plant': ['Brazil', 'Germany', 'India', 'Japan', 'Mexico', 'U.S.'],
    'fc_p': [20, 45, 14, 13, 30, 23],
    'fc_h': [ 5, 13,  3,  4,  6,  5],
    'fc_r': [ 5, 13,  3,  4,  6,  5],
    'rm_h': [3.6, 3.9, 3.6, 3.9, 3.6, 3.6],
    'pc_h': [5.1, 6.0, 4.5, 6.0, 5.0, 5.0],
    'rm_r': [4.6, 5.0, 4.5, 5.1, 4.6, 4.5],
    'pc_r': [6.6, 7.0, 6.0, 7.0, 6.5, 6.5],
})
pcosts.set_index('plant', inplace=True)

tcosts = pd.DataFrame({
    'from': ['Brazil', 'Germany', 'India', 'Japan', 'Mexico', 'U.S.'],
    'LatinAmerica': [ 0.20, 0.45, 0.50, 0.50, 0.40, 0.45],
    'Europe':       [ 0.45, 0.20, 0.35, 0.40, 0.30, 0.30],
    'AsiaWoJapan':  [ 0.50, 0.35, 0.20, 0.30, 0.50, 0.45],
    'Japan':        [ 0.50, 0.40, 0.30, 0.10, 0.45, 0.45],
    'Mexico':       [ 0.40, 0.30, 0.50, 0.45, 0.20, 0.25],
    'U.S.':           [ 0.45, 0.30, 0.45, 0.45, 0.25, 0.20],
})
tcosts.set_index('from', inplace=True)

duties = pd.DataFrame({
    'from': ['LatinAmerica', 'Europe', 'AsiaWoJapan', 'Japan', 'Mexico', 'U.S.'],
    'duty': [ 0.30, 0.03, 0.27, 0.06, 0.35, 0.04],
})
duties.set_index('from', inplace=True)

# Your provided exchange_rate_data
exrate0 = {
    '2018': [3.88, 4.33, 69.63, 109.91, 19.64, 1],
    '2019': [4.33, 0.92, 71.48, 109.82, 18.65, 1],
    '2020': [5.19, 0.82, 73.66, 103.24, 19.90, 1],
    '2021': [5.26, 0.88, 74.28, 115.59, 20.62, 1],
    '2022': [5.29, 0.93, 82.75, 131.12, 19.48, 1],
    '2023': [4.85, 0.91, 83.04, 140.99, 16.96, 1],
}
exrate0 = pd.DataFrame(exrate0 , index=['BRL', 'EUR', 'INR', 'JPY', 'MXN', 'USD'])

Drawing Samples

In [ ]:
!pip install forex_python

from forex_python.converter import CurrencyRates
from datetime import datetime, timedelta

import numpy as np
from scipy.stats import multivariate_normal

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 4.4 MB/s eta 0:00:00


In [ ]:
def get_monthly_exchange_rates(start_year, end_year, base_currency, target_currencies):
    c = CurrencyRates()
    rates = {currency: [] for currency in target_currencies}

    # Generate a list of first day of each month in the specified range
    start_date = datetime(start_year, 1, 1)
    end_date = datetime(end_year, 12, 31)
    current_date = start_date

    while current_date <= end_date:
        for currency in target_currencies:
            try:
                # Get exchange rate for the first day of the month
                rate = c.get_rate(base_currency, currency, current_date)
                rates[currency].append((current_date.strftime("%Y-%m"), rate))
            except Exception as e:
                print(f"Error fetching rate for {currency} on {current_date.strftime('%Y-%m')}: {e}")
                rates[currency].append((current_date.strftime("%Y-%m"), None))

        # Move to the first day of the next month
        if current_date.month == 12:
            current_date = datetime(current_date.year + 1, 1, 1)
        else:
            current_date = datetime(current_date.year, current_date.month + 1, 1)

    return rates

In [ ]:
# Specify the currencies
base_currency = "USD"
target_currencies = ["BRL", "EUR", "INR", "JPY", "MXN"]

# Fetch exchange rates from Jan 2019 to Dec 2023
exchange_rates = get_monthly_exchange_rates(2019, 2023, base_currency, target_currencies)

# Print the rates
for currency, rates in exchange_rates.items():
    print(f"Exchange rates for {currency} against {base_currency}:")
    for date, rate in rates:
        print(f"{date}: {rate}")
    print("\n")


Exchange rates for BRL against USD:
2019-01: 3.8812227074235808
2019-02: 3.6709964257693315
2019-03: 3.7808134938065536
2019-04: 3.8771804912780348
2019-05: 3.9267249064004286
2019-06: 3.9872657160792757
2019-07: 3.8272094457661465
2019-08: 3.8255866630424937
2019-09: 4.157212758245742
2019-10: 4.164433841071756
2019-11: 3.989316814794865
2019-12: 4.230468038608632
2020-01: 4.0196724230016025
2020-02: 4.266829533116178
2020-03: 4.485014120433634
2020-04: 5.2440563277249455
2020-05: 5.384792203015815
2020-06: 5.3324937027707815
2020-07: 5.442857142857142
2020-08: 5.167032410533423
2020-09: 5.433052473512972
2020-10: 5.6008339006126615
2020-11: 5.779363993845102
2020-12: 5.311915106951871
2021-01: 5.193953223046206
2021-02: 5.442320423700762
2021-03: 5.579025968638513
2021-04: 5.631619274646687
2021-05: 5.346548584671412
2021-06: 5.202126789366054
2021-07: 4.960871760350051
2021-08: 5.1060465898578755
2021-09: 5.152407548447152
2021-10: 5.415172413793104
2021-11: 5.650889618241493
2021-1

In [ ]:
# Convert the data into a NumPy array for analysis
# Note: This example assumes all currencies have the same number of rates and ignores missing values

# Extract rates for each currency, ensuring they're in the same order for each month
rates_list = [rates for rates in exchange_rates.values() if all(rate is not None for _, rate in rates)]
currency_data = np.array([[rate for _, rate in currency_rates] for currency_rates in rates_list])

# Transpose to get rows as observations (dates) and columns as variables (currencies)
currency_data = currency_data.T

# Ensure no NaN values; this method requires complete data
if np.isnan(currency_data).any():
    print("Data contains NaN values. Please handle missing data before proceeding.")
else:
    # Estimate the mean and covariance
    mean_vector = np.mean(currency_data, axis=0)
    covariance_matrix = np.cov(currency_data, rowvar=False)

    # Fit the multivariate normal distribution
    mvn_distribution = multivariate_normal(mean=mean_vector, cov=covariance_matrix)

    print("Mean Vector:\n", mean_vector)
    print("Covariance Matrix:\n", covariance_matrix)
    # The distribution is now defined and can be used for further analysis

Mean Vector:
 [  4.91679403   0.89791993  75.7301565  119.26608249  19.81262121]
Covariance Matrix:
 [[ 3.42594387e-01 -2.13708474e-03  1.22952065e+00  1.27836101e+00
   3.46111090e-01]
 [-2.13708474e-03  2.17002114e-03  1.24913080e-01  5.21137097e-01
  -1.73246912e-02]
 [ 1.22952065e+00  1.24913080e-01  2.02929685e+01  6.06245515e+01
  -2.53203458e+00]
 [ 1.27836101e+00  5.21137097e-01  6.06245515e+01  2.26189966e+02
  -1.36106797e+01]
 [ 3.46111090e-01 -1.73246912e-02 -2.53203458e+00 -1.36106797e+01
   2.22048686e+00]]


In [ ]:
num_draws = 100
df = {}

# Generate samples and store in a dictionary
samples = mvn_distribution.rvs(size=num_draws, random_state=42)
for i, sample in enumerate(samples, start=1):
    sample_list = sample.tolist()
    sample_list.append(1)
    df[i] = sample_list

exrate0 = pd.DataFrame(df, index=['BRL', 'EUR', 'INR', 'JPY', 'MXN', 'USD'])

In [ ]:
exrate0

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
BRL,5.339033,5.610633,4.158135,4.231257,4.221797,4.202571,5.522542,4.544914,4.819163,4.968826,...,5.448185,4.677103,4.745664,5.129649,4.148266,6.213829,4.396904,5.186395,5.357669,4.076926
EUR,0.866244,0.929650,0.914179,0.913227,0.860101,0.910053,0.940841,0.926353,0.844138,0.903790,...,0.858786,0.890685,0.811998,0.912408,0.904640,0.886242,0.859546,0.923140,0.935326,0.866398
INR,73.119811,79.345561,76.785541,76.098870,69.368990,73.048904,81.672869,81.905187,73.082681,77.420104,...,78.081892,79.188651,67.998090,77.518865,73.090580,77.527642,75.069506,80.662290,78.511699,71.729788
JPY,111.976947,122.173290,126.464114,128.222328,97.334656,118.178479,127.448130,137.209372,108.066423,130.463048,...,119.624349,131.070092,89.611745,122.416274,124.239082,122.822677,121.384236,140.793295,118.709868,111.603781
MXN,20.647650,21.555964,19.359696,18.853919,21.080869,19.383593,20.769920,17.194056,20.517260,19.746639,...,19.587135,18.120138,20.947551,20.808291,16.326318,20.152830,17.492399,16.972230,21.078569,19.358894
USD,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


# Minimize cost using Gurobi Binary and Integer optimizer

## Functions to calculate cost, unmet demand, and excess capacity

In [ ]:
# identify number of supply and demand location for iterations
n_ctry = range(demand.shape[0])
n_lines = range(demand.shape[1]+1)

# Objective function to calculate cost
def calc_total_cost(dec_plant, dec_h, dec_r, base_yr=1, selected_yr=2023, tariff=0):
    x_plant = np.array(list(dec_plant.values())).reshape(len(n_ctry), len(n_lines))
    x_h = np.array(list(dec_h.values())).reshape(len(n_ctry), len(n_ctry))
    x_r = np.array(list(dec_r.values())).reshape(len(n_ctry), len(n_ctry))

    # Adjust the cost using exchange rate of give year
    reindx = exrate0.loc[:, base_yr] / exrate0.loc[:, selected_yr]

    pcosts_rev = pcosts.values * reindx.values.reshape(-1,1)
    pcosts_rev = pd.DataFrame(pcosts_rev, columns=pcosts.columns[0:], index=pcosts.index)
    # return pcosts_rev
    # pcosts_rev = adj_pcosts_exrate(2019, 2023)

    duties_mat = np.zeros(len(duties)) + (1 + duties['duty'].values)[:, np.newaxis]
    np.fill_diagonal(duties_mat, 1)
    duties_mat = pd.DataFrame(duties_mat.T, index=pcosts_rev.index, columns=duties.index)
    duties_mat.loc['Germany', 'U.S.'] += tariff
    duties_mat.loc['U.S.', 'Europe']  += tariff

    vcosts_h = tcosts.add(pcosts_rev['rm_h'], axis=0).add(pcosts_rev['pc_h'], axis=0) * duties_mat
    vcosts_r = tcosts.add(pcosts_rev['rm_r'], axis=0).add(pcosts_rev['pc_r'], axis=0) * duties_mat

    fc = pcosts_rev[['fc_p','fc_h','fc_r']].values
    vh = (vcosts_h * x_h).values
    vr = (vcosts_r * x_r).values
    total_cost = sum(0.2 * fc[i,j] for i in n_ctry for j in n_lines) + sum(0.8 * fc[i,j] * x_plant[i,j] for i in n_ctry for j in n_lines) + sum(vh[i,j] for i in n_ctry for j in n_ctry) + sum(vr[i,j] for i in n_ctry for j in n_ctry)

    return total_cost


# Calculate excess capacity given decision variables
def calc_excess_cap(dec_plant, dec_h, dec_r):
    x_plant = np.array(list(dec_plant.values())).reshape(len(n_ctry), len(n_lines))
    x_h = np.array(list(dec_h.values())).reshape(len(n_ctry), len(n_ctry))
    x_r = np.array(list(dec_r.values())).reshape(len(n_ctry), len(n_ctry))

    excess_cap = (x_plant * caps.values).copy()
    excess_cap[:, 0] -= (np.sum(x_h, axis=1) + np.sum(x_r, axis=1))
    excess_cap[:, 1] -= np.sum(x_h, axis=1)
    excess_cap[:, 2] -= np.sum(x_r, axis=1)
    return excess_cap

# Calculate unmet demand given decision variables
def calc_unmet_demand(dec_h, dec_r):
    x_h = np.array(list(dec_h.values())).reshape(len(n_ctry), len(n_ctry))
    x_r = np.array(list(dec_r.values())).reshape(len(n_ctry), len(n_ctry))

    x_h_sum = np.sum(x_h, axis=0)
    x_r_sum = np.sum(x_r, axis=0)
    unmet_demand = (demand.values).copy()
    unmet_demand = np.column_stack((x_h_sum - unmet_demand[:, 0], x_r_sum - unmet_demand[:, 1]))

    return unmet_demand


## Gurobi optimizer

In [ ]:
tariff = 0

from collections import Counter

# Initialize the dictionary to count plant openings
plant_openings = {country: 0 for country in n_ctry}

# Initialize a Counter object to track strategies
strategy_counter = Counter()

# Initialize a list to save the results of each simulation
results = []

for year in range(1, 101):
  selected_yr = year

  # Create a Gurobi model
  model = Model("MinimizeCost")

  # Assign initial value of decision variables
  dec_plant = {(i, j): 1 for i in n_ctry for j in n_lines}
  dec_h     = {(i, j): 1 for i in n_ctry for j in n_ctry}
  dec_r     = {(i, j): 1 for i in n_ctry for j in n_ctry}

  # Define decision variables
  dec_plant = {(i, j): model.addVar(vtype=GRB.BINARY, name=f"Dec_plant_{i}_{j}")    for i in n_ctry for j in n_lines}
  dec_h     = {(i, j): model.addVar(vtype=GRB.CONTINUOUS, lb=0, name=f"Dec_h_{i}_{j}") for i in n_ctry for j in n_ctry}
  dec_r     = {(i, j): model.addVar(vtype=GRB.CONTINUOUS, lb=0, name=f"Dec_r_{i}_{j}") for i in n_ctry for j in n_ctry}

  # Excess Capacity constraints
  excess_cap = calc_excess_cap(dec_plant, dec_h, dec_r)
  for i in n_ctry:
      for j in n_lines:
          model.addConstr(excess_cap[i, j] >= 0, name=f"Excess_Cap_Constraints_{i}_{j}")


  # Unmet demand constraints
  unnmet_demand = calc_unmet_demand(dec_h, dec_r)
  for i in n_ctry:
      for j in range(2):
          model.addConstr(unnmet_demand[i,j] == 0, name=f"Unmet_Demand_Constraints_{i}_{j}")


  # Update the model
  model.update()

  # Set objective function - Total cost = Fixed cost + Variable costs of Highcal and Relax lines
  model.setObjective(calc_total_cost(dec_plant, dec_h, dec_r, base_yr, selected_yr, tariff), GRB.MINIMIZE)

  # Suppress optimization output
  model.Params.OutputFlag = 0

  # Optimize the model
  model.optimize()

# Extract results to print as table
  op_plant = pd.DataFrame([[dec_plant[i, j].X for j in n_lines] for i in n_ctry], columns=['Plant', 'H', 'R'], index=n_ctry)
  op_h = pd.DataFrame([[dec_h[i, j].X for j in n_ctry] for i in n_ctry], columns=n_ctry, index=n_ctry)
  op_r = pd.DataFrame([[dec_r[i, j].X for j in n_ctry] for i in n_ctry], columns=n_ctry, index=n_ctry)

# Create a dictionary for this year's strategy
  yearly_strategy = {}
  for country in n_ctry:
      h_status = 'Open' if op_plant.at[country, 'H'] > 0.5 else 'Closed'
      r_status = 'Open' if op_plant.at[country, 'R'] > 0.5 else 'Closed'
      country_status = 1 if (h_status == 'Open' or r_status == 'Open') else 0
      yearly_strategy[country] = {'C': country_status, 'H': h_status, 'R': r_status}
      if op_plant.at[country, 'Plant'] > 0.5:  # Assuming 'Plant' indicates if any plant is open
          plant_openings[country] += 1

# Convert the strategy to a string and update the counter
  strategy_str = ', '.join([f"{country}('C':{country_status},H:{h_status}, R:{r_status})" for country, (country_status, h_status, r_status) in yearly_strategy.items()])
  strategy_counter[strategy_str] += 1

  # Append the yearly strategy to the results list
  results.append(yearly_strategy)

# Print the total counts for each country
print("Total Plant Openings per Country over 100 Simulations:")
for country, count in plant_openings.items():
    print(f"{country}: {count}")

# Find the three most common strategies
most_common_strategies = strategy_counter.most_common(3)

# Print the three most common strategies
print("\nThree Most Common Strategies:")
for i, (strategy, frequency) in enumerate(most_common_strategies, start=1):
    print(f"Strategy {i}:")
    print(f"{strategy} - Frequency: {frequency}")

Restricted license - for non-production use only - expires 2025-11-24
Total Plant Openings per Country over 100 Simulations:
0: 86
1: 100
2: 100
3: 49
4: 69
5: 83

Three Most Common Strategies:
Strategy 1:
0('C':C,H:H, R:R), 1('C':C,H:H, R:R), 2('C':C,H:H, R:R), 3('C':C,H:H, R:R), 4('C':C,H:H, R:R), 5('C':C,H:H, R:R) - Frequency: 100


Brazil: 56
Germany: 100
India: 100
Japan: 24
Mexico: 81
US: 97

In [ ]:
results

[{0: {'C': 1, 'H': 'Open', 'R': 'Open'},
  1: {'C': 1, 'H': 'Closed', 'R': 'Open'},
  2: {'C': 1, 'H': 'Open', 'R': 'Open'},
  3: {'C': 0, 'H': 'Closed', 'R': 'Closed'},
  4: {'C': 1, 'H': 'Open', 'R': 'Open'},
  5: {'C': 1, 'H': 'Open', 'R': 'Open'}},
 {0: {'C': 1, 'H': 'Open', 'R': 'Open'},
  1: {'C': 1, 'H': 'Closed', 'R': 'Open'},
  2: {'C': 1, 'H': 'Open', 'R': 'Open'},
  3: {'C': 1, 'H': 'Open', 'R': 'Open'},
  4: {'C': 1, 'H': 'Open', 'R': 'Open'},
  5: {'C': 0, 'H': 'Closed', 'R': 'Closed'}},
 {0: {'C': 0, 'H': 'Closed', 'R': 'Closed'},
  1: {'C': 1, 'H': 'Closed', 'R': 'Open'},
  2: {'C': 1, 'H': 'Open', 'R': 'Open'},
  3: {'C': 1, 'H': 'Closed', 'R': 'Open'},
  4: {'C': 1, 'H': 'Open', 'R': 'Open'},
  5: {'C': 1, 'H': 'Open', 'R': 'Closed'}},
 {0: {'C': 0, 'H': 'Closed', 'R': 'Closed'},
  1: {'C': 1, 'H': 'Closed', 'R': 'Open'},
  2: {'C': 1, 'H': 'Open', 'R': 'Open'},
  3: {'C': 1, 'H': 'Closed', 'R': 'Open'},
  4: {'C': 1, 'H': 'Open', 'R': 'Open'},
  5: {'C': 1, 'H': 'Open

In [ ]:
# Extracting 'C' values for each scenario and creating a DataFrame
df_data = {i: {k: v['C'] for k, v in scenario.items()} for i, scenario in enumerate(results)}
df = pd.DataFrame(df_data).T  # Transpose to get scenarios as rows

df

,0,1,2,3,4,5
0,1,1,1,0,1,1
1,1,1,1,1,1,0
2,0,1,1,1,1,1
3,0,1,1,1,1,1
4,0,1,1,0,1,1
...,...,...,...,...,...,...
95,1,1,1,0,1,1
96,1,1,1,1,0,1
97,1,1,1,1,0,1
98,1,1,1,0,1,0


In [ ]:
# Calculate the correlation matrix
correlation_matrix = df.corr()

correlation_matrix
# Define the new column names
new_column_names = ['Brazil', 'Germany', 'India', 'Japan', 'Mexico', 'US']

# Check if the number of new column names matches the number of columns in the DataFrame
if len(new_column_names) == len(df.columns):
    # Assign new column names
    df.columns = new_column_names
else:
    print("The number of new column names does not match the number of columns in the DataFrame.")

# Display the updated DataFrame
df

,Brazil,Germany,India,Japan,Mexico,US
0,1,1,1,0,1,1
1,1,1,1,1,1,0
2,0,1,1,1,1,1
3,0,1,1,1,1,1
4,0,1,1,0,1,1
...,...,...,...,...,...,...
95,1,1,1,0,1,1
96,1,1,1,1,0,1
97,1,1,1,1,0,1
98,1,1,1,0,1,0


In [ ]:
correlation_matrix = df.corr()

correlation_matrix

,Brazil,Germany,India,Japan,Mexico,US
Brazil,1.000000,NaN,NaN,0.280182,-0.270440,-0.182600
Germany,NaN,NaN,NaN,NaN,NaN,NaN
India,NaN,NaN,NaN,NaN,NaN,NaN
Japan,0.280182,NaN,NaN,1.000000,-0.683822,-0.248697
Mexico,-0.270440,NaN,NaN,-0.683822,1.000000,-0.303348
US,-0.182600,NaN,NaN,-0.248697,-0.303348,1.000000


In [ ]:
#strongest correlation is between Mexico and Japan -0.683822

In [ ]:

# Helper function to convert 'Open'/'Closed' to 1/0
def status_to_num(status):
    return 1 if status == 'Open' else 0

# We need to flatten the results since they are nested dictionaries
flat_results = []
for result in results:
    flat_result = {}
    for country_data in result.values():
        for line, status in country_data.items():
            # Combine the country index and line name to create a unique key
            flat_result[line] = status_to_num(status)
    flat_results.append(flat_result)

germany_h_statuses = [result['H'] for result in flat_results]
japan_h_statuses = [result['H'] for result in flat_results]

# Calculate the Pearson correlation coefficient for the HighCal line
correlation_h = np.corrcoef(germany_h_statuses, japan_h_statuses)[0, 1]

# Do the same for the Relax line
germany_r_statuses = [result['R'] for result in flat_results]
japan_r_statuses = [result['R'] for result in flat_results]

# Calculate the Pearson correlation coefficient for the Relax line
correlation_r = np.corrcoef(germany_r_statuses, japan_r_statuses)[0, 1]

print(f"The correlation between Germany and Japan plant statuses for HighCal is: {correlation_h}")
print(f"The correlation between Germany and Japan plant statuses for Relax is: {correlation_r}")



The correlation between Germany and Japan plant statuses for HighCal is: 1.0
The correlation between Germany and Japan plant statuses for Relax is: 1.0


In [ ]:
# Create a dictionary of the strategies
strategies = {
    "Strategy": ["1", "2", "3"],
    0: ["Closed, Closed", "Open, Open", "Open, Open"],
    1: ["Closed, Open", "Closed, Open", "Closed, Open"],
    2: ["Open, Open", "Open, Open", "Open, Open"],
    3: ["Closed, Closed", "Closed, Closed", "Closed, Closed"],
    4: ["Open, Open", "Open, Open", "Open, Open"],
    5: ["Open, Closed", "Open, Open", "Open, Closed"],
    "Frequency": [40, 19, 15]
}

# Convert the dictionary to a DataFrame
strategies_df = pd.DataFrame(strategies)

# Correct the loop to create separate tables for each strategy
strategy_tables = []

# Loop through each strategy
for i in strategies['Strategy']:
    # Create a new DataFrame for each strategy
    df = pd.DataFrame({
        'Plant': range(6),
        'H': [value.split(', ')[0] for value in strategies_df.loc[strategies_df['Strategy'] == i, 0:5].values.flatten()],
        'R': [value.split(', ')[1] for value in strategies_df.loc[strategies_df['Strategy'] == i, 0:5].values.flatten()]
    })
    strategy_tables.append((i, df))

# Convert the list of tuples to a dictionary for easy access
strategy_tables_dict = {f"Strategy {num}": df for num, df in strategy_tables}
strategy_tables_dict



{'Strategy 1':    Plant       H       R
 0      0  Closed  Closed
 1      1  Closed    Open
 2      2    Open    Open
 3      3  Closed  Closed
 4      4    Open    Open
 5      5    Open  Closed,
 'Strategy 2':    Plant       H       R
 0      0    Open    Open
 1      1  Closed    Open
 2      2    Open    Open
 3      3  Closed  Closed
 4      4    Open    Open
 5      5    Open    Open,
 'Strategy 3':    Plant       H       R
 0      0    Open    Open
 1      1  Closed    Open
 2      2    Open    Open
 3      3  Closed  Closed
 4      4    Open    Open
 5      5    Open  Closed}

In [ ]:
#aligns with previous correlation pairs observation

# Question 3

In [ ]:
# Generate exchange rate samples and store in a dictionary
num_draws = 100
df = {}

np.random.seed(100)

samples = mvn_distribution.rvs(size=num_draws)
for i, sample in enumerate(samples, start=1):
    sample_list = sample.tolist()
    sample_list.append(1)
    df[i] = sample_list

# Create DataFrame directly from samples, initially with 5 columns
exrate0 = pd.DataFrame(samples, columns=['BRL', 'EUR', 'INR', 'JPY', 'MXN'])

# Add an extra column 'Base' with all values set to 1
exrate0['USD'] = 1

print(exrate0.head())
print(exrate0.shape)


        BRL       EUR        INR         JPY        MXN  USD
0  5.275032  1.000918  83.115236  145.584488  19.575040    1
1  4.817317  0.867400  74.436394  111.269621  19.488896    1
2  5.375109  0.902145  78.576238  125.870200  19.173449    1
3  4.626499  0.907107  74.824238  121.232451  20.257444    1
4  5.202501  0.845789  72.085768   94.175486  22.278629    1
(100, 6)


In [ ]:
selected_yr = 2023
base_yr = 1

demand = pd.DataFrame({
    'from': ['LatinAmerica', 'Europe', 'AsiaWoJapan', 'Japan', 'Mexico', 'U.S.'],
    'd_h': [  7, 15,  5,  7,  3, 18],
    'd_r': [  7, 12,  3,  8,  3, 17],
})
demand.set_index('from', inplace=True)

caps = pd.DataFrame({
    'plant': ['Brazil', 'Germany', 'India', 'Japan', 'Mexico', 'U.S.'],
    'cap': [18, 45, 18, 10, 30, 22],
})
caps.set_index('plant', inplace=True)

pcosts = pd.DataFrame({
    'plant': ['Brazil', 'Germany', 'India', 'Japan', 'Mexico', 'U.S.'],
    'fc_p': [20, 45, 14, 13, 30, 23],
    'fc_h': [ 5, 13,  3,  4,  6,  5],
    'fc_r': [ 5, 13,  3,  4,  6,  5],
    'rm_h': [3.6, 3.9, 3.6, 3.9, 3.6, 3.6],
    'pc_h': [5.1, 6.0, 4.5, 6.0, 5.0, 5.0],
    'rm_r': [4.6, 5.0, 4.5, 5.1, 4.6, 4.5],
    'pc_r': [6.6, 7.0, 6.0, 7.0, 6.5, 6.5],
})
pcosts.set_index('plant', inplace=True)

tcosts = pd.DataFrame({
    'from': ['Brazil', 'Germany', 'India', 'Japan', 'Mexico', 'U.S.'],
    'LatinAmerica': [ 0.20, 0.45, 0.50, 0.50, 0.40, 0.45],
    'Europe':       [ 0.45, 0.20, 0.35, 0.40, 0.30, 0.30],
    'AsiaWoJapan':  [ 0.50, 0.35, 0.20, 0.30, 0.50, 0.45],
    'Japan':        [ 0.50, 0.40, 0.30, 0.10, 0.45, 0.45],
    'Mexico':       [ 0.40, 0.30, 0.50, 0.45, 0.20, 0.25],
    'U.S.':           [ 0.45, 0.30, 0.45, 0.45, 0.25, 0.20],
})
tcosts.set_index('from', inplace=True)

duties = pd.DataFrame({
    'from': ['LatinAmerica', 'Europe', 'AsiaWoJapan', 'Japan', 'Mexico', 'U.S.'],
    'duty': [ 0.30, 0.03, 0.27, 0.06, 0.35, 0.04],
})
duties.set_index('from', inplace=True)

In [ ]:
def calc_total_cost_new(dec_plant, dec_h, dec_r, exrate):
    global pcosts, tcosts, duties, caps, demand
    n_ctry = len(demand)  # Number of countries or plants

    # Ensure dec_h and dec_r dictionaries are populated correctly to match expected dimensions
    # Example correction without detailed context on how dec_h and dec_r are populated:
    x_plant = np.array(list(dec_plant.values())).reshape(1, 6)  # Assuming this remains as is for binary decisions
    x_h = np.array(list(dec_h.values())).reshape(1, 6)
    x_r = np.array(list(dec_r.values())).reshape(1, 6)


    # Use 'USD' column as the current or alternative year's rate for cost adjustment
    reindx = exrate['USD'].reshape(-1, 1)  # Using 'USD' column directly

    pcosts_rev = pcosts.values * reindx
    pcosts_rev = pd.DataFrame(pcosts_rev, columns=pcosts.columns, index=pcosts.index)

    duties_mat = np.zeros((n_ctry, n_ctry)) + (1 + duties['duty'].values)[:, np.newaxis]
    np.fill_diagonal(duties_mat, 1)
    duties_mat = pd.DataFrame(duties_mat, index=pcosts_rev.index, columns=duties.index)

    vcosts_h = tcosts.add(pcosts_rev['rm_h'], axis=0).add(pcosts_rev['pc_h'], axis=0) * duties_mat
    vcosts_r = tcosts.add(pcosts_rev['rm_r'], axis=0).add(pcosts_rev['pc_r'], axis=0) * duties_mat

    fc = pcosts_rev[['fc_p','fc_h','fc_r']].values
    vh = (vcosts_h.values * x_h).sum()
    vr = (vcosts_r.values * x_r).sum()
    total_cost = sum(0.2 * fc[:,j] * x_plant[:,j] for j in range(3)) + vh + vr

    return total_cost


In [ ]:
def status_to_binary(status):
    return 1 if status == "Open" else 0

def simulate_strategy(strategy_df, exrate_samples):
    costs = []
    for _, exrate_sample in exrate_samples.iterrows():
        dec_plant = strategy_df['Plant'].apply(status_to_binary).to_dict()
        dec_h = strategy_df['H'].apply(status_to_binary).to_dict()
        dec_r = strategy_df['R'].apply(status_to_binary).to_dict()
        # Directly using global data structures within calc_total_cost_new
        total_cost = calc_total_cost_new(dec_plant, dec_h, dec_r, exrate_sample)
        costs.append(total_cost)
    return costs

In [ ]:
strategy_costs = {}
for name, strategy_df in strategy_tables_dict.items():
    costs = simulate_strategy(strategy_df, exrate0)
    mean_cost, std_dev_cost = np.mean(costs), np.std(costs)
    strategy_costs[name] = {'Mean Cost': mean_cost, 'Standard Deviation of Cost': std_dev_cost}

# Display results
for name, costs in strategy_costs.items():
    print(f"Strategy: {name}, Mean Cost: {costs['Mean Cost']}, Standard Deviation: {costs['Standard Deviation of Cost']}")


Strategy: Strategy 1, Mean Cost: 429.4184999999999, Standard Deviation: 1.1368683772161603e-13
Strategy: Strategy 2, Mean Cost: 653.0154999999999, Standard Deviation: 1.1368683772161603e-13
Strategy: Strategy 3, Mean Cost: 571.4695, Standard Deviation: 1.1368683772161603e-13
